In [ ]:
# For each player:
    # Select all matches and timelines that they are in
    # Identify champion played
    # Identify role played
    # Main role = most played role where role is played > 40% of the time
    # Main champion = most played champion
    
    # For each match:
        # Get DMG share
        # Get DMG Taken share
        # Get 0-10 CSPM
        # Get 10-20 CSPM
        # Get 20+ CSPM:
            # Subtract CS at 20 from total CS, divide by game time 
    
    # For each timeline:
        # Get number of duel kills and deaths
        # Get number of teamfight kills and deaths
        # Get number of skirmish kills and deaths
        # Get 0-10 K+D+A
        # Get 10-20 K+D+A
        # Get 20+ K+D+A

Questions to answer:
*We want to use ratios not absolute stats, as we're comparing high-elo to low-elo so stuff like CSPM is just not going to be useful...*

#### Will need later:
- Most-played champion
- Most-played role

#### Fighting fingerprint: percentage of fights with are duels, skirmishes, and teamfights...
- Can I calculate duels (1v1 fights as detected by the death of one or both players)
- Stat contribution: easy. Get DMG Share and DMG Taken Share
- Can I calculate teamfights (3v3+ fights as detected by assists or kills)
- Can I calculate picks (1 player dying to 2 or more players with no return kill (how to define return kill?)
- Can I calculate skirmishes (2vX)

#### Farming to Fighting Ratio
Early Game: pre-10 CSPM / average(kills + assists + deaths) before 10  
Mid Game: 10-20 min CSPM / average(kills + assists + deaths) between 10 and 20  
Late Game: 20+ CSPM / average(kills + assists + deaths) after 20  

#### Lane aggressiveness:
CSD and XPD @ 15   
average (kills + assists + deaths) before 15    
(STRETCH) relate to number of roams?    

#### Vision focus:
Wards placed pre-10  
Wards placed pre-20  
Wards placed pre-30  

Vision Score/Minute   

Wards Cleared  
Pinks purchased  
(STRETCH) % of enemy wards cleared  

#### Aggression
Average time to First Kill/Assist  
Average time to First Tower  
% of kills = solo kills  
Forward Kills: % of kills past enemy 1st turret line  
Kill Participation  
Overall Kills+Assists / Minute  

#### Teamwork
% of team's objective damage
% of team's turret damage  
\# of dragons killed / game  
\# of barons killed / game
average time to first baron





In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import jsonlines
import json

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
def ms_to_s(ms):
    return round(ms / 1000)

In [ ]:
na_matches = []
with jsonlines.open('../data/matches_na.jsonl') as infile:
    for line in infile:
        na_matches.append(line)
        
        

In [ ]:
na_timelines = []
with jsonlines.open('../data/timelines_na.jsonl') as infile:
    for line in infile:
        na_timelines.append(line)

In [ ]:
ranked_matches = []
ranked_timelines = []
is_ranked = []
game_ids = []
for i, match in enumerate(na_matches):
    if match['queueId'] == 420 and match['gameId'] not in game_ids:
        ranked_matches.append(match)
        is_ranked.append(i)
        game_ids.append(match['gameId'])
        
for i, timeline in enumerate(na_timelines):
    if i in is_ranked:
        ranked_timelines.append(timeline)
        
del na_matches
del na_timelines
del is_ranked
del game_ids


In [ ]:
all_players = []

for match in ranked_matches:
    for player in match['participantIdentities']:
        all_players.append(player['player']['summonerId'])

players_to_evaluate = pd.Series(all_players).value_counts().reset_index(name="count").query("count > 50")["index"]


In [ ]:
[[(i, event) for event in frame['events'] if event['type'] == 'CHAMPION_KILL' and event['killerId'] == 3] for i, frame in enumerate(ranked_timelines[0]['frames'])]

In [ ]:
ranked_timelines[0]['frames'][11]['events']

In [ ]:
# CSD and XPD @ 15   
# average (kills + assists + deaths) before 15    

ranked_matches[0]['participants']

We want to make sure we are incorporating only full games, where all players were connected and playing, so we will take a look at the distribution of maximum attained level in games which ended between 15 and 16 minutes (you can only initiate a surrender vote at the 15-minute mark). Based on this graph, we will not consider any games in which a player failed to exceed level 7 as valid.

In [ ]:
short_matches = [match for match in ranked_matches if match['gameDuration'] / 60 < 16 and match['gameDuration'] / 60 > 15]
levels = []
for match in short_matches:
    for participant in match['participants']:
        levels.append(participant['stats']['champLevel'])

plt.figure(figsize=(8,5))
sns.distplot(levels, bins=12, kde=False)
plt.xlabel('Max level attained')
plt.ylabel('Count')
plt.title('Distribution of Maximum Level Attained in 15-16 minute games')
plt.xticks(range(1,13));

In [ ]:
players_to_evaluate[0]

In [ ]:
all_players.head()

In [ ]:
ranked_matches[8]